<a href="https://colab.research.google.com/github/Arseniy16/Computational_Math/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа 2
###  *Численное решение СЛАУ*
---

Написать программу для численного решения СЛАУ произвольного
порядка. Предусмотреть возможность проверки совместности системы,
достаточных условий применимости используемых методов, оценки числа обусловленности.
1. Реализовать метод прогонки для трехдиагональной матрицы СЛАУ.
2. Реализовать методы Якоби и Зейделя итерационного решения СЛАУ.
3. Реализовать методы наискорейшего спуска и наименьшей невязки.


---


Реализуем метод прогонки для трехдиагональной матрицы СЛАУ.

Для этого необходимо:


1.   Соответствие размерностей
2.   Условие диагонального преобладания
3.   Ненулевые элементы на главной диагонали ($a_{ii} \neq 0 $)


Определение прогоночных коэф-ов:

$
    v_i = \dfrac{a_{[i] [i+1]}}{-a_{[i][i]} - a_{[i][i-1]} \cdot v_{i-1}}
$

$
    u_i = \dfrac{  a_{[i][i-1]} \cdot u_{i-1} - b_i }{ -a_{[i][i]} - a_{[i][i-1]} \cdot v_{i-1}   }
$

\\

Для первой строки матрицы:

$ v_0 = \dfrac{ a_{[0][1]} }{ a_{[0][0]} } $ и $ u_0 = \dfrac{ - b_0 }{ -a_{[0][0]} }$

\\

Для последней строки матрицы:

$ v_{n-1} = 0 $

$ u_{n-1} = \dfrac{ a_{[n-1][n-2]} \cdot u_{n-2} - b_{n-1} }{ -a_{[n-1][n-1]} - a_{[n-1][n-2]} \cdot v_{n-2} }$


In [199]:
import numpy as np
from numpy import linalg


In [200]:
# Ax = B
N = 5
rand_num = 50

buf = np.zeros((N, N), dtype = 'int')

flat = buf.ravel()
flat[0::N+1] = np.random.randint(0, rand_num, N)
flat[N::N+1] = np.random.randint(0, rand_num, N-1)
flat[1::N+1] = np.random.randint(0, rand_num, N-1)

A = buf
B = np.random.randint(rand_num, size=(N, 1))

In [201]:
# Проверка на диагональное преобладание
if (np.abs(A[0][0]) < np.abs(A[0][1]) ): 
    A[0][0] = np.abs(A[0][1]) + np.random.randint(1, 5)

if (np.abs(A[N-1][N-1]) < np.abs(A[N-1][N-2]) ): 
    A[N-1][N-1] = np.abs(A[N-1][N-2]) + np.random.randint(1,5)

for i in range(1, N-1):
    if(np.abs(A[i][i]) < np.abs(A[i][i-1]) + np.abs(A[i][i+1]) ): 
        A[i][i] = np.abs(A[i][i-1]) + np.abs(A[i][i+1]) + np.random.randint(1,10)  

A_B = np.hstack((A, B))
# print(B.flatten())
print('Матрица A:\n', A)
print('Матрица B:\n', B)
print('Расширенная матрица:\n', A_B)

# print(A, '\n\n', A_B) #debug

Матрица A:
 [[11  7  0  0  0]
 [ 6 19  9  0  0]
 [ 0 28 59 29  0]
 [ 0  0  1 32 23]
 [ 0  0  0 39 42]]
Матрица B:
 [[36]
 [47]
 [28]
 [ 2]
 [43]]
Расширенная матрица:
 [[11  7  0  0  0 36]
 [ 6 19  9  0  0 47]
 [ 0 28 59 29  0 28]
 [ 0  0  1 32 23  2]
 [ 0  0  0 39 42 43]]


Проверим совместность СЛАУ, используя Теорему Кронекера-Капелли.

In [202]:
rkA = linalg.matrix_rank(A)
rkB = linalg.matrix_rank(A_B)
print('Ранг A = ', rkA, '\nРанг A|B = ', rkB) 
if (rkA == rkB): print('СЛАУ - совместна')
else: print('СЛАУ - несовместна')

Ранг A =  5 
Ранг A|B =  5
СЛАУ - совместна


In [ ]:
'''
# for test(don't use)

# Пробные данные для уравнения A*X = B
a = [[ 10.8000, 0.0475,      0, 0     ],
     [  0.0321, 9.9000, 0.0523, 0     ],
     [       0, 0.0369, 9.0000, 0.0570],
     [       0,      0, 0.0416, 8.1000]]
 
b = [12.1430, 13.0897, 13.6744, 13.8972]
 
# Решение, которое должно получиться:
# x1 = 1,118587
# x2 = 1,310623
# x3 = 1,503186
# x4 = 1,707983
'''

Реализуем метод прогонки для решения СЛАУ.

In [204]:
import math 

# Вывод матрицы на экран
def print_arr( string, namevec, a ):
    if (type(a) == int) or (type(a) == float):
        print(a)
    else:
        print( string )
        for k in range(len(a)):   
            print("{}[{}] = {:8.4f}".format(namevec, k, a[k]))
            
 
# Проверка 3х-диаг. матрицы коэффициентов на корректность
def isCorrectArray(a):
    n = len(a)
    
    for row in range(0, n):
        if( len(a[row]) != n ):
            print('Не соответствует размерность')
            return False
        
    for row in range(1, n - 1):
        if(abs(a[row][row]) < abs(a[row][row - 1]) + abs(a[row][row + 1])):
            print('Не выполнены условия достаточности')
            return False

    if (abs(a[0][0]) < abs(a[0][1])) or (abs(a[n - 1][n - 1]) < abs(a[n - 1][n - 2])):
        print('Не выполнены условия достаточности')
        return False
        
    
    for row in range(0, len(a)):
        if( a[row][row] == 0 ):
            print('Нулевые элементы на главной диагонали')
            return False
    return True
 
  
# Процедура нахождения решения 3-х диагональной матрицы
def solution_runThrough(a, b):
    if( not isCorrectArray(a) ):
        print('Ошибка в исходных данных')
        return -1 
 
    n = len(a)
    x = [0 for k in range(0, n)] # обнуление вектора решений
    print('Размерность матрицы: ',n,'x',n)
    
    # Прямой ход
    v = [0 for k in range(0, n)]
    u = [0 for k in range(0, n)]
    # для первой 0-й строки
    v[0] = a[0][1] / (-a[0][0]) 
    u[0] = ( - b[0]) / (-a[0][0]) 
    for i in range(1, n - 1): # заполняем за исключением 1-й и (n-1)-й строк матрицы
        v[i] = a[i][i+1] / ( -a[i][i] - a[i][i-1]*v[i-1] )
        u[i] = ( a[i][i-1]*u[i-1] - b[i] ) / ( -a[i][i] - a[i][i-1]*v[i-1] )
    # для последней (n-1)-й строки
    v[n-1] = 0
    u[n-1] = (a[n-1][n-2]*u[n-2] - b[n-1]) / (-a[n-1][n-1] - a[n-1][n-2]*v[n-2])
    
    print_arr('Прогоночные коэффициенты v: ','v', v)
    print_arr('Прогоночные коэффициенты u: ','u', u)
    
    # Обратный ход
    x[n-1] = u[n-1]
    for i in range(n-1, 0, -1):
        x[i-1] = v[i-1] * x[i] + u[i-1]
        
    return x    
                

Реализуем методы Якоби и Зейделя итерационного решения СЛАУ.

In [205]:
import copy
 
# Проверка матрицы коэффициентов на корректность
def isCorrectArray_new(a, b):
    for row in range(0, len(a)):        
        if( len(a[row]) != len(b) ):
            print('Не соответствует размерность')
            return False
    
    for row in range(0, len(a)):
        if( a[row][row] == 0 ):
            print('Нулевые элементы на главной диагонали')
            return False
    return True
 
 
# Условие завершения программы  
def isNeedToComplete(x_old, x_new):
    eps = 0.0001
    sum_up = 0
    # sum_low = 0
    for k in range(0, len(x_old)):
        sum_up += ( x_new[k] - x_old[k] ) ** 2

        # sum_low += ( x_new[k] ) ** 2
        
    # return math.sqrt( sum_up / sum_low ) < eps
    return math.sqrt( sum_up ) < eps    


# Далее реализуем каждый из методов
# ---------------------------------
# метод Якоби
def solution_jacobi(a, b):

    if( not isCorrectArray_new(a, b) ):
        print('Ошибка в исходных данных')
    else:
        count = len(b) # количество корней
        
        x = [1 for k in range(0, count) ] # начальное приближение корней
        
        numberOfIter = 0  # подсчет количества итераций
        MAX_ITER = 100    # максимально допустимое число итераций

        while( numberOfIter < MAX_ITER ):
 
            x_prev = copy.deepcopy(x)
            
            for k in range(0, count):
                S = 0
                for j in range(0, count):
                    if( j != k ): S = S + a[k][j] * x_prev[j]
                
                x[k] = b[k]/a[k][k] - S / a[k][k]   
            
            if isNeedToComplete(x_prev, x) : 
                break
              
            numberOfIter += 1
 
        print('Количество итераций на решение: ', numberOfIter)
        
        return x

# метод Зейделя
def solution_zeidel(a, b):

    if( not isCorrectArray_new(a, b) ):
        print('Ошибка в исходных данных')
    else:
        count = len(b) # количество корней
        
        x = [1 for k in range(0, count) ] # начальное приближение корней
        
        numberOfIter = 0  # подсчет количества итераций
        MAX_ITER = 100    # максимально допустимое число итераций

        while( numberOfIter < MAX_ITER ):
 
            x_prev = copy.deepcopy(x)
            
            for k in range(0, count):
                S = 0
                for j in range(0, count):
                    if( j != k ): S = S + a[k][j] * x[j] 
                x[k] = b[k]/a[k][k] - S / a[k][k]
            
            if isNeedToComplete(x_prev, x) : 
                break
              
            numberOfIter += 1
 
        print('Количество итераций на решение: ', numberOfIter)
        
        return x  

# метод наискорейшего спуска
def solution_fastDescent(a, b):

    if( not isCorrectArray_new(a, b) ):
        print('Ошибка в исходных данных')
    else:
        count = len(b)

        x = [1 for k in range(0, count) ] # начальное приближение корней
        x = [b[i] / a[i][i] for i in range(0, count) ] # начальное приближение корней


        numberOfIter = 0  # подсчет количества итераций
        MAX_ITER = 100   # максимально допустимое число итераций

        while( numberOfIter < MAX_ITER ):

            x_prev = copy.deepcopy(x)

            r = np.dot(a, x) - b

            tau =  np.dot(r, r) / np.dot(np.dot(a, r), r)

            x = x - tau * r

            if isNeedToComplete(x_prev, x) : 
                break
              
            numberOfIter += 1
 
        print('Количество итераций на решение: ', numberOfIter)

        return x
    
# метод минимальных невязок
def solution_minError(a, b):

    if( not isCorrectArray_new(a, b) ):
        print('Ошибка в исходных данных')
    else:
        count = len(b)

        x = [1 for k in range(0, count) ] # начальное приближение корней
        x = [b[i] / a[i][i] for i in range(0, count) ] # начальное приближение корней

        numberOfIter = 0  # подсчет количества итераций
        MAX_ITER = 100   # максимально допустимое число итераций

        while( numberOfIter < MAX_ITER ):

            x_prev = copy.deepcopy(x)

            r = np.dot(a, x) - b

            Ar = np.dot(a, r)

            tau =  np.dot(Ar, r) / np.dot(Ar, Ar)

            x = x - tau * r

            if isNeedToComplete(x_prev, x) : 
                break
              
            numberOfIter += 1
 
        print('Количество итераций на решение: ', numberOfIter)

        return x


А теперь протестируем все методы.

In [206]:
# метод прогонки
method_run_through = solution_runThrough(A, B.flatten())
print_arr('Решение: ','x', method_run_through)

Размерность матрицы:  5 x 5
Прогоночные коэффициенты v: 
v[0] =  -0.6364
v[1] =  -0.5928
v[2] =  -0.6839
v[3] =  -0.7344
v[4] =   0.0000
Прогоночные коэффициенты u: 
u[0] =   3.2727
u[1] =   1.8024
u[2] =  -0.5299
u[3] =   0.0808
u[4] =   2.9835
Решение: 
x[0] =   2.4704
x[1] =   1.2608
x[2] =   0.9136
x[3] =  -2.1104
x[4] =   2.9835


In [207]:
# метод Якоби
method_jacobi = solution_jacobi(A.tolist(), list(B.flatten()) )
print( 'Метод Якоби:\nРешение: ',  method_jacobi)

Количество итераций на решение:  56
Метод Якоби:
Решение:  [2.470320890808543, 1.2608719760855418, 0.9134534199684177, -2.110392810312383, 2.9834036705417644]


In [208]:
# метод Зейделя
method_zeidel = solution_zeidel(A.tolist(), list(B.flatten()) )
print( 'Метод Зейделя:\nРешение: ',  method_zeidel)

Количество итераций на решение:  27
Метод Зейделя:
Решение:  [2.4702928331375302, 1.260925082572672, 0.9134526178986337, -2.1103661884119753, 2.9834352701920723]


In [209]:
# метод наискорейшего спуска
method_fastDescent = solution_fastDescent(A.tolist(), list(B.flatten()) )
print( 'Метод наискорейшего спуска:\nРешение: ',  method_fastDescent)

Количество итераций на решение:  51
Метод наискорейшего спуска:
Решение:  [ 2.47007196  1.26107582  0.91334999 -2.11036304  2.9834086 ]


In [210]:
# метод минимальных невязок
method_minErr = solution_minError(A.tolist(), list(B.flatten()) )
print( 'Метод минимальных невязок:\nРешение: ',  method_minErr)

Количество итераций на решение:  23
Метод минимальных невязок:
Решение:  [ 2.46990535  1.26140327  0.91294351 -2.10981129  2.98283054]


Оценим число обусловленности СЛАУ.

In [211]:
mu = np.linalg.norm(np.linalg.inv(A)) * np.linalg.norm(B) / np.linalg.norm(method_run_through) 
print('Число обусловленности СЛАУ:', mu)

Число обусловленности СЛАУ: 4.446621224656659
